# Q22 Quick Start
### **Introduction to S&P500 stocks**

This template demonstrates how to build a trading strategy, specifically designed for use with a dataset containing historical constituents of the S&P 500. The strategy can only trade stocks that are part of the S&P 500 index, and only when the stock is considered liquid. A company is deemed liquid if it is included in the S&P 500 at the observed point in time.

The strategy takes a low-risk approach by allocating weights to stocks based on their volatility. It assigns capital only when the ratio of the Average True Range (ATR(14)) to the closing price is below 0.0205, signaling lower volatility. The allocation is further refined by the stock's transaction money flow, where stocks with higher liquidity and trading volume receive a larger portion of the capital (weight). This approach ensures that more capital is directed towards assets with higher market activity, while also maintaining a risk-aware allocation.

**Important Considerations:**

- Trade only on historical S&P 500 stocks, and only when they are considered liquid (i.e., part of the index at the time of observation).
- The in-sample period begins on 2006-01-01. Earlier data may be used for testing and training purposes.
- The strategy must achieve a minimum in-sample Sharpe Ratio of 0.70 to be considered valid.
- The maximum allocation to any single asset is capped at 10% of total capital. If a weight exceeds this, it will be limited to 0.1.
- Manual stock selection or direct hand-picking is not permitted. The allocation process must be [automatic](https://quantiacs.com/documentation/en/user_guide/dynamic_assets_selection.html).
- The strategy can open both long and short positions.

For official Q22 rules, click [here](https://quantiacs.com/contest/22).





### **Accessing the S&P 500 Stocks Dataset - Data loading**

For the Q22 contest, a new dataset of S&P 500 stocks is available. You can obtain this dataset by calling the function stocks_load_spx_data() from the data module:

```python
snp_stocks_data = qndata.stocks_load_spx_data(min_date='2005-06-01')
```

To check the list of S&P 500 companies starting from 2006-01-01, along with additional stock information, use the load_list() function:


```python
snp_stocks_list = qndata.stocks_load_spx_list()
```

**Quantiacs** also provides data for other instruments, such as [cryptocurrencies](https://quantiacs.com/documentation/en/data/crypto.html), [indices](https://quantiacs.com/documentation/en/data/indexes.html), [futures](https://quantiacs.com/documentation/en/data/futures.html), as well as additional [fundamental stock](https://quantiacs.com/documentation/en/data/fundamental.html) data and [macroeconomic](https://quantiacs.com/documentation/en/data/bls.html) data. Note that there is also a [Nasdaq-100](https://quantiacs.com/documentation/en/data/stocks.html#stocks-nasdaq100) dataset available, and it is not a subset of the S&P 500. However, there is a significant overlap between the two.


The data is provided in xarray.DataArray format. Check [here](https://quantiacs.com/documentation/en/data/intro.html#working-with-data) for more details on manipulating [xarray](https://quantiacs.com/documentation/en/data/intro.html#xarray) data.



**Technical analysis - indicators**

Once the data is loaded, various technical indicators from the qnt.ta module can be used to generate trading signals. For a complete list of indicators and examples of their implementation, check the [documentation](https://quantiacs.com/documentation/en/user_guide/technical_indicators.html) page.

### **Trading strategy - algorithm**

In [13]:
# Necessary imports
import xarray as xr
import numpy as np

import qnt.stats as qnstats
import qnt.data as qndata
import qnt.output as qnout
import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.graph as qngraph



def strategy(data, state=None, mfsp=135, limit=0.0205): # parameter state is used in the backtester (multi-pass) for stateful strategies
    vol = data.sel(field="vol")  ### Volume values
    liq = data.sel(field="is_liquid")  ### Liquidity values, 1.0 or 0.0 -> True or False
    close = data.sel(field="close")  ### close prices
    high = data.sel(field="high")  ### daily high
    low = data.sel(field="low")  ### daily low

    atrs = qnta.atr(high=high, low=low, close=close, ma=14)  ### Average True Range of 14 bars (working days)
    ratio = atrs / close  ### indicator
    weights = xr.where(ratio > limit, 0, 1)  ### Strategy condition - zero weight if ratio is bigger than limit

    money_vol = vol * liq * close  ### Daily money flow per liquid asset
    total_money_vol = money_vol.sum(dim='asset', skipna=True)  ### Total daily money flow of liquid assets
    money_vol_share = money_vol / total_money_vol  ### Daily money flow share per liquid asset
    mvs_mov = qnta.sma(money_vol_share, mfsp)  ### weights allocation by average money flow share in period of 135 days

    return mvs_mov * weights


#### **Weights - Single / Multi pass approach**

In [Single](https://quantiacs.com/documentation/en/user_guide/functional_quality.html#single-pass-backtesting) pass backtesting, which is significantly faster, weights are calculated using the entire dataset in a single run. 

[Multi](https://quantiacs.com/documentation/en/user_guide/functional_quality.html#multi-pass-backtesting) pass backtesting evaluates weights on a day-by-day basis by slicing the dataset for each individual day.


If applicable, we recommend using the single pass approach for efficiency, while verifying strategy statistics with the multi pass backtester. If the statistics from single pass and multi pass match exactly, it indicates that forward-looking bias has likely not been introduced, and the strategy can be confidently submitted as single pass.

In rare cases, even if results are identical between single and multi pass, forward-looking bias might unintentionally occur (e.g., by incorporating global data variables into the logic). Such issues are generally mitigated in production but can result in discrepancies in statistics comparing development and production results.


In [14]:
### SINGLE PASS ###

## The min_date parameter is set to '2005-06-01' to ensure that the dataset includes at least 135 bars (approximately 190 days) of data,
## which is necessary for creating indicators and determining weights for the strategy.
snp_stocks_data = qndata.stocks_load_spx_data(min_date='2005-06-01')

weights = strategy(snp_stocks_data)
weights = qnout.clean(weights, snp_stocks_data) # fix liquidity

fetched chunk 1/19 0s
fetched chunk 2/19 0s
fetched chunk 3/19 0s
fetched chunk 4/19 0s
fetched chunk 5/19 0s
fetched chunk 6/19 0s
fetched chunk 7/19 0s
fetched chunk 8/19 1s
fetched chunk 9/19 1s
fetched chunk 10/19 1s
fetched chunk 11/19 1s
fetched chunk 12/19 1s
fetched chunk 13/19 1s
fetched chunk 14/19 1s
fetched chunk 15/19 1s
fetched chunk 16/19 1s
fetched chunk 17/19 1s
fetched chunk 18/19 1s
fetched chunk 19/19 1s
Data loaded 2s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
WARNING! Strategy trades non-liquid assets.
Fix liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.




When using the backtester for weight calculation, there is no need to manually load data or define a "load_data" function as input. It is only required to specify the competition_type (in this case, "stocks_s&p500"), and the corresponding data will be loaded automatically by default.



In [15]:
### MULTI PASS ###

w=qnbt.backtest(
    competition_type="stocks_s&p500",
    lookback_period=250,
    start_date="2006-01-01",
    strategy=strategy,
    analyze=True,
 )

#### **Statistics**

While the multi pass approach automatically displays the strategy’s statistics through the backtester, single pass mode only calculates the weights, so visualizing the stats must be done manually. You can use the helper print_stats function for this purpose:


In [27]:
def print_stats(stat):
    display(stat.sel(time=slice('2006-01-01', None)).to_pandas().tail(10))

    performance = stat.to_pandas()["equity"]
    qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

def get_sharpe(stat):
    return stat.isel(time=-1).sel(field="sharpe_ratio").item()


In [16]:
stat = qnstats.calc_stat(snp_stocks_data, weights.sel(time=slice('2006-01-01', None)))
print_stats(stat)

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2024-08-27,2.121163,0.000412,0.047415,-0.002032,-0.074607,0.868636,0.041187,1.0,680.0,0.029167,27.632917
2024-08-28,2.120525,-0.000301,0.047410,-0.002332,-0.074607,0.868183,0.041161,1.0,680.0,0.029164,27.632201
2024-08-29,2.123048,0.001190,0.047406,-0.001145,-0.074607,0.869477,0.041218,1.0,680.0,0.029162,27.631993
2024-08-30,2.128105,0.002382,0.047404,0.000000,-0.074607,0.872133,0.041342,1.0,680.0,0.029158,27.631286
2024-09-03,2.124293,-0.001791,0.047401,-0.001791,-0.074607,0.869885,0.041233,1.0,680.0,0.029154,27.630820
2024-09-04,2.124954,0.000311,0.047396,-0.001481,-0.074607,0.870155,0.041242,1.0,680.0,0.029156,27.620654
2024-09-05,2.120725,-0.001990,0.047393,-0.003468,-0.074607,0.867664,0.041121,1.0,680.0,0.029154,27.618309
2024-09-06,2.115784,-0.002330,0.047392,-0.005790,-0.074607,0.864757,0.040982,1.0,680.0,0.029151,27.614809
2024-09-09,2.122710,0.003274,0.047392,-0.002535,-0.074607,0.868409,0.041156,1.0,680.0,0.029151,27.607511


In [17]:
get_sharpe(stat)

0.8700874342573661

#### **Submit strategy to the competition**

> Use **Submit** button on my strategies page

Make sure that qnout.write(weights) has been added to cell, and the weights have been written. It is not required when using Multi pass backtester.

**Note:** After submitting the strategy to the contest, any weight exceeding 0.1 will be capped at that limit. You can apply weight normalization functions before submission, which may be more suitable, such as those that maintain the ratio between allocated weights. Details on these functions can be found [here](https://quantiacs.com/documentation/en/user_guide/dynamic_assets_selection.html#risk-management).

In [18]:
### Run this cell to get all weights below 0.1. This "hard cut" way is used on production if any weight exceeds 0.1.
### Any other normalization method can be used.
import qnt.exposure as qnexp

weights_capped = qnexp.cut_big_positions(weights=weights, max_weight=0.1)

Compare weights before and after normalization. Check the statistics again, it can change not only in negative way. 

In [ ]:
qnout.write(weights_capped)

### **Common Reasons for Submission Rejection and Their Solutions**

Here are some of the frequent reasons causing submission rejection in algorithmic trading competitions, and their corresponding remedies.

> [Detailed explanation with examples](https://quantiacs.com/documentation/en/user_guide/passFilters.html).

**1) Missed call to write_output**

Save algorithm weights,  run code

```python
qnt.output.write(weights)
```

**2) Not eligible send to contest. In-Sample Sharpe must be larger than 0.7**

Improve your algorithm. For example, you can use sections and get an algorithm that will pass the filter

* Example [Trading System Optimization](https://github.com/quantiacs/strategy-futures-ta-global-optimizer/blob/master/strategy.ipynb)
* Example of a strategy using [technical analysis indicators](https://github.com/quantiacs/strategy-predict-NASDAQ100-use-atr-lwma/blob/master/strategy.ipynb)

**Need help?** Check the [**Documentation**](https://quantiacs.com/documentation/en/) and find solutions/report problems in the [**Forum**](https://quantiacs.com/community/categories) section.


**3) Not enough bid information.**

Run code

```python
min_time = weights.time[abs(weights).fillna(0).sum('asset')> 0].min()
min_time
```
> min_time must be less than or equal to January 1, 2006.

If min_time is larger than the starting date, we recommend to fill the starting values of the time series with non-vanishing values, for example a simple buy-and-hold strategy.

```python
def get_enough_bid_for(data_, weights_):
    time_traded = weights_.time[abs(weights_).fillna(0).sum('asset') > 0]
    is_strategy_traded = len(time_traded)
    if is_strategy_traded:
        return xr.where(weights_.time < time_traded.min(), data_.sel(field="is_liquid"), weights_)
    return weights_


weights_new = get_enough_bid_for(data, weights)
weights_new = weights_new.sel(time=slice("2006-01-01",None))
```